# Paraphrase Detection: How to run inference on the endpoint you have created?

First, we import some packages the we will need to call our endpoint and process our input/output.

In [ ]:
import json
import boto3
import numpy as np
from scipy.special import softmax

Now, we simulate an input user `question` and define the list of `faqs` that we want to match that question against. You can see in the example below that these are standard questions that might apply to any subscribtion or order-based service, much like Amazon! 

------
## 1. Prepare the input

In [ ]:
question = 'Where can I change my contact information?'

In [ ]:
faqs = [
    'How can I change my contact details?',
    'Can I change my subscription status at any time of the month?',
    'How should I handle missing deliveries?',
    'What is the return policy for orders?',
    'Will I have access to extra services and products?'
]

We now create a combination list of question/hypothesis for each FAQ, which the our deployd model will take as input.

In [ ]:
pairs = [[question,faq] for faq in faqs]

In [16]:
[print(pair) for idx,pair in enumerate(pairs)]

['Where can I change my contact information?', 'How can I change my contact details?']
['Where can I change my contact information?', 'Can I change my subscription status at any time of the month?']
['Where can I change my contact information?', 'How should I handle missing deliveries?']
['Where can I change my contact information?', 'What is the return policy for orders?']
['Where can I change my contact information?', 'Will I have access to extra services and products?']


[None, None, None, None, None]

------
## 2. Query the endpoint that you have created


We are now going to query the endpoint we have created, get paraphrase predictions for every FAQ, and figure out if any of them (if so, which one) means the same as our user's question!

First, let's define some helper functions:

- `query_endpoint` uses the Sagemaker Runtime's boto3 client to invoke the endpoint, passing it your encoded input
- `parse_response` parses and decodes the reponse from your endpoint, grabbing the relevant keys from the model output
- `compute_softmax` is responsible for computing the softmax function over the logits output by the model, turning them into meaningful 0-1 probabilities for each possible label (0 or 1)

<div class="alert alert-block alert-info">
<b>Note:</b> If you have set a custom name for your endpoint, don't forget to change the value of the <b>your_endpoint</b> variable in the next cell
</div>

In [ ]:
global your_endpoint
your_endpoint =  'jumpstart-ftc-hf-spc-distilroberta-base'

In [ ]:
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
def query_endpoint(encoded_text):
    endpoint_name = your_endpoint
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/list-text', Body=encoded_text, Accept='application/json;verbose')
    return response

def parse_response(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    logits, predicted_label, labels = model_predictions['probabilities'], model_predictions['predicted_label'], model_predictions['labels']
    return logits, predicted_label, labels

def compute_softmax(logs):
    logits_array = np.array(logs)
    probs = list(softmax(logits_array))
    return probs 

Finally, we loop over all the question/FAQ combinations, sending them to our model for prediction. 

In [15]:
for idx, sentence_pair in enumerate(pairs):
    query_response = query_endpoint(json.dumps(sentence_pair).encode('utf-8'))
    logits, predicted_label, labels = parse_response(query_response)
    probabilities = compute_softmax(logits)
    text_label = 'Matching question!' if predicted_label == 1 else 'No relationship'
    print (f"Prediction for pair {idx}:{newline}"
            f"question: {sentence_pair[0]}{newline}"
            f"faq: {sentence_pair[1]}{newline}"
            f"model prediction: {probabilities}{newline}"
            f"predicted label: {bold}{predicted_label}{unbold}{newline}"
            f"predicted label: {bold}{text_label}{unbold}{newline}")

Prediction for pair 0:
question: Where can I change my contact information?
faq: How can I change my contact details?
model prediction: [0.0014380110406731113, 0.9985619889593269]
predicted label: 1
predicted label: Matching question!

Prediction for pair 1:
question: Where can I change my contact information?
faq: Can I change my subscription status at any time of the month?
model prediction: [0.9987948971475786, 0.001205102852421532]
predicted label: 0
predicted label: No relationship

Prediction for pair 2:
question: Where can I change my contact information?
faq: How should I handle missing deliveries?
model prediction: [0.9989790234083225, 0.0010209765916776399]
predicted label: 0
predicted label: No relationship

Prediction for pair 3:
question: Where can I change my contact information?
faq: What is the return policy for orders?
model prediction: [0.9989804913506942, 0.0010195086493056586]
predicted label: 0
predicted label: No relationship

Prediction for pair 4:
question: Wher

As we can see, all question pairs are correctly classified, with only the first pair being identified as a paraphrase with almost 98% certainty!

In fact, we can qualitatively confirm that `Where is the page to change my contact information?` represents the same question as `How can I change my contact details?`